In [0]:
storage_account_name = "clickstreamdlstorage"
adls_container_name = "clickstream-datalake"
mount_point = "/mnt/clickstream_datalake"

In [0]:
client_id = "631748e7-fd26-4169-8997-16d57362a171"
tenant_id = "28f91947-f927-412e-a9c2-6fc359f63f2c"
client_secret = dbutils.secrets.get(scope="datalake-secrets", key="adls_sp_client_secret")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

def mount_adls_container(container_name, mount_point):

    if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        try:
            dbutils.fs.mount(
                source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
                mount_point = mount_point,
                extra_configs = configs
            )
            print(f"Successfully mounted {mount_point}")
        except Exception as e:
            print(f"Failed to mount {mount_point}: {e}")
    else:
        print(f"{mount_point} is already mounted.")


mount_adls_container(adls_container_name, mount_point)

In [0]:
connection_string = dbutils.secrets.get(scope="eventhubs-secrets", key="eventhub-connection-string")
eventhub_name = dbutils.secrets.get(scope="eventhubs-secrets", key="eventhub-name")
encrypted_connection_string = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connection_string)

ehConf = {
    'eventhubs.connectionString': encrypted_connection_string,
    'eventhubs.eventHubName': eventhub_name,
    'eventhubs.consumerGroup': '$Default'
}




In [0]:
bronze_checkpoint_location = f"{mount_point}/eventhub_checkpoints/clickstream_bronze/{eventhub_name}"
bronze_layer_path = f"{mount_point}/raw_clickstream_delta/bronze/{eventhub_name}"

silver_checkpoint_location = f"{mount_point}/eventhub_checkpoints/clickstream_silver/{eventhub_name}"
silver_layer_path = f"{mount_point}/processed_clickstream_delta/silver/{eventhub_name}"

gold_checkpoint_location = f"{mount_point}/eventhub_checkpoints/clickstream_gold/{eventhub_name}"
gold_layer_path = f"{mount_point}/curated_clickstream_delta/gold/{eventhub_name}"
